Url: https://tbrain.trendmicro.com.tw/Competitions/Details/2

In [1]:
#import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql import Row
from pyspark.sql.functions import col, udf, lag, rank, lit
from pyspark.sql.window import Window

In [2]:
global Path
if sc.master[0:5]=="local":
    #Path = "file:/c:/D Drive/work/bigData/pySpark/TBrain_Round2_DataSet_20180427"
    #Path = "file:/Users/yungchuanlee/Documents/learn/AI競賽/ETF預測/TBrain_Round2_DataSet_20180427"
    Path = "file:/home/hduser/app/bigdata/competition/etf/TBrain_Round2_DataSet_20180427"
else:
    Path = "hdfs://master:9000/user/hduser"

In [3]:
sc.master

'local[*]'

In [4]:
#define alias of columns
col_alias_etf= {"代碼":"etf_id", "日期": "etf_date", "中文簡稱": "etf_name", "開盤價(元)":"etf_open", 
            "最高價(元)":"etf_high", "最低價(元)":"etf_low", "收盤價(元)":"etf_close", "成交張數(張)":"etf_count"}
col_alias_stock= {"代碼":"stock_id", "日期": "stock_date", "中文簡稱": "stock_name", "開盤價(元)":"stock_open", 
            "最高價(元)":"stock_high", "最低價(元)":"stock_low", "收盤價(元)":"stock_close", "成交張數(張)":"stock_count"}

In [5]:
#udf
def to_double(str_val):
    return float(str_val.replace(",",""))
to_double=udf(to_double)

In [6]:
#def function to read data (因檔案格式都相同)
def read_data(file_name, col_alias):
    str_cols = ["代碼","日期", "中文簡稱"]
    raw_data = spark.read.option("encoding", "Big5").csv(Path + "/" + file_name, header=True, sep=",")
    print("Total " + file_name + " count: " + str(raw_data.count()))
    #rename cols and correct type 
    num_cols = [col_name for col_name in raw_data.columns if col_name not in str_cols]
    final_data=raw_data.select( [col(str_col_name).alias(col_alias[str_col_name]) for str_col_name in str_cols] + 
                                  [to_double(col(num_col_name)).cast("double").alias(col_alias[num_col_name]) for num_col_name in num_cols] )
    final_data.printSchema()
    final_data.show(5)
    return final_data

In [7]:
print("starting import tetfp.csv(台灣18檔ETF股價資料)...")
tetfp_dt=read_data("tetfp.csv", col_alias_etf)

starting import tetfp.csv(台灣18檔ETF股價資料)...
Total tetfp.csv count: 19053
root
 |-- etf_id: string (nullable = true)
 |-- etf_date: string (nullable = true)
 |-- etf_name: string (nullable = true)
 |-- etf_open: double (nullable = true)
 |-- etf_high: double (nullable = true)
 |-- etf_low: double (nullable = true)
 |-- etf_close: double (nullable = true)
 |-- etf_count: double (nullable = true)

+-------+--------+----------------+--------+--------+-------+---------+---------+
| etf_id|etf_date|        etf_name|etf_open|etf_high|etf_low|etf_close|etf_count|
+-------+--------+----------------+--------+--------+-------+---------+---------+
|0050   |20130102|元大台灣50          |    54.0|   54.65|   53.9|     54.4|  16487.0|
|0050   |20130103|元大台灣50          |    54.9|   55.05|  54.65|    54.85|  29020.0|
|0050   |20130104|元大台灣50          |   54.85|   54.85|   54.4|     54.5|   9837.0|
|0050   |20130107|元大台灣50          |   54.55|   54.55|   53.9|    54.25|   8910.0|
|0050   |20130108|元大台灣50     

In [8]:
#EDA
#range of date
tetfp_dt.describe('etf_date').show()

+-------+-------------------+
|summary|           etf_date|
+-------+-------------------+
|  count|              19053|
|   mean|2.015329328116307E7|
| stddev| 15717.456119401833|
|    min|           20130102|
|    max|           20180427|
+-------+-------------------+



In [9]:
# print("starting import taetfp.csv(台灣18檔ETF調整後股價資料)...")
# taetfp_dt=read_data("taetfp.csv", col_alias_etf)

In [10]:
# print("starting import tsharep.csv(台灣個股股價資料)...")
# tsharep_dt=read_data("tsharep.csv", col_alias_stock)

In [11]:
# print("starting import tasharep.csv(台灣個股調整後股價資料)...")
# tasharep_dt=read_data("tasharep.csv", col_alias_stock)

In [12]:
import sys
from pyspark.sql.functions import lag, col, avg,collect_list, lit
from pyspark.sql.window import Window
from pyspark.sql.types import ArrayType, DoubleType, IntegerType
#declare previous row windows
wsSpec_etf = Window.partitionBy('etf_id').orderBy('etf_date') #time window for normal case
wsSpec_etf_close_price_raw = Window.partitionBy('etf_id').orderBy('row_idx').rangeBetween(-sys.maxsize, -1)
wsSpec_etf_dif_raw = Window.partitionBy('etf_id').orderBy('row_idx').rangeBetween(-sys.maxsize, 0)
def avg_list(p_list):
    #計算數字list的平均值
    return sum(p_list)/len(p_list)
#計算EMA的udf
def calculate_ema_native(close_p_list, window_len):
    #透過歷史收盤價計算
    if len(close_p_list) < window_len:
        return None
    elif len(close_p_list) == window_len:
        #if len of list = win_len then return avg, 
        return avg_list(close_p_list)
    else:
        #else EMA[t] =(EMA[t-1]*(win_len-1)+close[t]*2)/(win_len+1)
        ema = avg_list(close_p_list[:window_len])
        for price in close_p_list[window_len:]:
            ema = (ema*(window_len-1)+price*2)/(window_len+1)
        return ema
calculate_ema=udf(calculate_ema_native, DoubleType())
#計算BIAS的udf
def calculate_bias(close_p_list):
    #計算前日收盤價與N日均線之差比: (close price - MA)/MA   ,Paper 建議用20日MA
    #因要預測今日的收盤價，故計算前日收盤價與前20日均線
    if len(close_p_list) < 21:
        return None
    else:
        list_len = len(close_p_list)
        p_close = close_p_list[-1]
        cal_list = close_p_list[list_len-21: list_len-1]
        return p_close - avg_list(cal_list)
calculate_bias=udf(calculate_bias, DoubleType())

def get_min_max_last(p_list):
    #找出list中最大最小和最後一個值, 回傳(min, max, last)
    return (min(p_list), max(p_list), p_list[-1])
def calculate_raw_rsv(p_list):
    #RSV = (收盤價-9日低值)/(9日高值-9日低值)
    p_min, p_max, p_last = get_min_max_last(p_list)
    rsv = (p_last - p_min)/(p_max - p_min)
    return rsv
def calculate_rsv(p_9_list, k_prev, d_prev):
    #計算加權後的RSV，p_9_list=>9日收盤價
    rrsv = calculate_raw_rsv(p_9_list)
    k_curr = (1/3)*rrsv + (2/3)*k_prev
    d_curr = (1/3)*k_curr + (2/3)*d_prev
    return [k_curr, d_curr]
#計算隨機指標（Stochastic Oscillator，KD），原名%K&%D
def calculate_KD(close_p_list):
    win_len = 9 #看過去 9 日值
    #RSV = (收盤價-9日低值)/(9日高值-9日低值)
    #K_curr = 1/3*RSV + 2/3*K_prev
    #D_curr = 1/3*K_curr + 2/3*D_prev
    if len(close_p_list) < win_len:
        return None
    elif len(close_p_list) == win_len:
        #無前日K, D時，以0.5帶入
        return calculate_rsv(close_p_list, 0.5, 0.5)
    else:
        kds = calculate_rsv(close_p_list[0:9], 0.5, 0.5)
        for idx in range(1, (len(close_p_list)+1-9)):
            p_9_list = close_p_list[idx: idx+9]
            kds = calculate_rsv(p_9_list, kds[0], kds[1])
        return kds
calculate_KD=udf(calculate_KD, ArrayType(DoubleType()))

#計算差離值DIF = 12日EMA - 26日EMA
def calculate_DIF_native(close_p_list):
    if len(close_p_list) < 26:
        return None
    else:
        ema12 = calculate_ema_native(close_p_list, 12)
        ema26 = calculate_ema_native(close_p_list, 26)
        return ema12 - ema26
calculate_DIF=udf(calculate_DIF_native, DoubleType())

#計算MACD=(前一日MACD × (9 - 1) + 今日DIF × 2) ÷ (9 + 1)
def calculate_MACD(dif_list, dif_curr):
    win_len = 9
    if len(dif_list) < win_len:
        return None
    elif len(dif_list) == win_len:
        #if len of list = win_len then return avg, 
        return avg_list(dif_list)
    else:
        #MACD=(前一日MACD × (9 - 1) + 今日DIF × 2) ÷ (9 + 1)
        macd = avg_list(dif_list[:win_len])
        for price in dif_list[win_len:]:
            macd = (macd*(win_len-1)+dif_curr*2)/(win_len+1)
        return macd
calculate_MACD=udf(calculate_MACD, DoubleType())

#計算相對強弱指數(RSI)
def calculate_RSI(close_p_list):
    win_len = 9
    if len(close_p_list) < (win_len + 1):
        return None
    else:
        cur_list = close_p_list[1:]
        prv_list = close_p_list[0:-1]
        p_dif_list = list(map(lambda x,y : x - y, cur_list, prv_list)) #dif list
        u_list = []
        d_list = []
        for dif in p_dif_list:
            if dif == 0:
                #若兩天價格相同，則U及D皆等於零
                u_list.append(0)
                d_list.append(0)
            elif dif > 0:
                #在價格上升的日子, U = diff, D = 0
                u_list.append(dif)
                d_list.append(0)
            else:
                #在價格下跌的日子, U = 0, D = abs(diff)
                u_list.append(0)
                d_list.append(abs(dif))
        #RSI = ema(u,9)/(ema(u,9)+ema(d,9))
        ema_u = calculate_ema_native(u_list, win_len)
        ema_d = calculate_ema_native(d_list, win_len)
        return ema_u/(ema_u + ema_d)
calculate_RSI=udf(calculate_RSI, DoubleType())

#計算威廉指標（Williams %R）
def calculate_WR(close_p_list):
    win_len = 9
    if len(close_p_list) < win_len:
        return None
    else:
        p_list = close_p_list[len(close_p_list) - win_len :]
        return 1.0 - calculate_raw_rsv(p_list)
calculate_WR=udf(calculate_WR, DoubleType())


In [13]:
#calculate ema [5,10,20] #cannot remove row_idx, row_idx for next window usage
tetfp_dt2=tetfp_dt.withColumn("row_idx", rank().over(wsSpec_etf)) \
    .withColumn("close_price_raw", collect_list(col('etf_close')).over(wsSpec_etf_close_price_raw)) \
    .withColumn("EMA5", calculate_ema(col("close_price_raw"), lit(5))) \
    .withColumn("EMA10", calculate_ema(col("close_price_raw"), lit(10))) \
    .withColumn("EMA20", calculate_ema(col("close_price_raw"), lit(20))) \
    .withColumn("BIAS", calculate_bias(col("close_price_raw"))) \
    .withColumn("KD", calculate_KD(col("close_price_raw"))) \
    .withColumn("K", col("KD")[0]).withColumn("D", col("KD")[1]) \
    .withColumn("DIF", calculate_DIF(col("close_price_raw"))) \
    .withColumn("dif_list", collect_list(col('DIF')).over(wsSpec_etf_dif_raw)) \
    .withColumn("MACD", calculate_MACD(col("dif_list"), col("DIF"))) \
    .withColumn("RSI", calculate_RSI(col("close_price_raw")))\
    .withColumn("WR", calculate_WR(col("close_price_raw")))

tetfp_dt2.cache()
tetfp_dt2.printSchema()

root
 |-- etf_id: string (nullable = true)
 |-- etf_date: string (nullable = true)
 |-- etf_name: string (nullable = true)
 |-- etf_open: double (nullable = true)
 |-- etf_high: double (nullable = true)
 |-- etf_low: double (nullable = true)
 |-- etf_close: double (nullable = true)
 |-- etf_count: double (nullable = true)
 |-- row_idx: integer (nullable = true)
 |-- close_price_raw: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- EMA5: double (nullable = true)
 |-- EMA10: double (nullable = true)
 |-- EMA20: double (nullable = true)
 |-- BIAS: double (nullable = true)
 |-- KD: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- K: double (nullable = true)
 |-- D: double (nullable = true)
 |-- DIF: double (nullable = true)
 |-- dif_list: array (nullable = true)
 |    |-- element: double (containsNull = true)
 |-- MACD: double (nullable = true)
 |-- RSI: double (nullable = true)
 |-- WR: double (nullable = true)



In [14]:
# tetfp_dt2.filter("etf_id='0050   '").select("row_idx", "etf_close", "MACD", "RSI", "WR") \
#             .show(45)
#          .toPandas().to_csv(Path.replace("file:","") + "/taetfp_BIAS.csv",index=False)

In [15]:
#匯出成pandas
# etf_pd_50 = tetfp_dt2.filter("etf_id='0050   '").select("row_idx", "etf_date" ,"etf_close", "EMA5", "EMA10", "EMA20", "BIAS") \
#                 .toPandas()

In [16]:
#圖形化顯示
#aetf_pd_50.set_index('etf_date') #set_index 後配合 loc select 出區段資料來看
# etf_pd_50_part = etf_pd_50.loc[1200:]
# etf_pd_50_part.etf_close.plot(x='row_idx', y='etf_close', style='b--', label="etf_close")
# etf_pd_50_part.EMA5.plot(x='row_idx', y='EMA5', label="EMA5", style='r-')
# etf_pd_50_part.EMA10.plot(x='row_idx', y='EMA10', label="EMA10", style='g-')
# etf_pd_50_part.EMA20.plot(x='row_idx', y='EMA20', label="EMA20", style='y-')
# plt.legend()
# plt.show()

In [17]:
#計算各欄位與收盤價之相關性
# corr_cols = ['EMA5','EMA10','EMA20','BIAS','K','D']
# for col in corr_cols:
#     print('corr between ', col , ' and etf_close: ', str(tetfp_dt2.corr(col, 'etf_close')))

In [18]:
from pyspark.ml.feature import MinMaxScaler, StandardScaler
from pyspark.ml.linalg import Vectors
tot_dt = tetfp_dt2.filter("MACD is not null") \
    .select("etf_id", "etf_date", "row_idx", "EMA5", "EMA10", "EMA20", "BIAS", "K", "D", "DIF", "MACD", "RSI", "WR", "etf_close") \
    .orderBy("etf_id", "etf_date", ascending=True)
    
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
#將Feature合併為Vector 並作標準化
assembler = VectorAssembler(
    inputCols=["EMA5", "EMA10", "EMA20", "BIAS", "K", "D", "DIF", "MACD", "RSI", "WR"],
    outputCol="features")
tot_dt_1 = assembler.transform(tot_dt)
minmax_scaler = MinMaxScaler(inputCol="features", outputCol="stdFeatures")
scaler_model = minmax_scaler.fit(tot_dt_1)
#std_scaler = StandardScaler(inputCol="features", outputCol="stdFeatures")
#scaler_model = std_scaler.fit(tot_dt_1)
tot_dt_scale = scaler_model.transform(tot_dt_1)
#tot_dt_scale=tot_dt_1.withColumn("stdFeatures", col("features")) #測試不作標準化

In [19]:
next_date_range = ["20180423", "20180424", "20180425","20180426", "20180427"]
predict_start_date = next_date_range[0]
#取出4/16~4/27 (共兩週資料作為測試集)
train_dt = tot_dt_scale.filter("etf_date < '" + predict_start_date + "' and MACD is not null") \
    .select("etf_id", "etf_date", "row_idx", "stdFeatures", "etf_close") \
    .orderBy("etf_id", "etf_date", ascending=True)
test_dt = tot_dt_scale.filter("etf_date >= '" + predict_start_date + "'") \
    .select("etf_id", "etf_date", "row_idx", "stdFeatures", "etf_close") \
    .orderBy("etf_id", "etf_date", ascending=True)
print('train count: ', str(train_dt.count()), ', test count: ', str(test_dt.count()))
train_dt.show(10)
test_dt.show(10)

train count:  18351 , test count:  90
+-------+--------+-------+--------------------+---------+
| etf_id|etf_date|row_idx|         stdFeatures|etf_close|
+-------+--------+-------+--------------------+---------+
|0050   |20130227|     35|[0.58940415111937...|     55.2|
|0050   |20130301|     36|[0.58819098072642...|     55.4|
|0050   |20130304|     37|[0.58824157226986...|    54.75|
|0050   |20130305|     38|[0.58548234159790...|     55.2|
|0050   |20130306|     39|[0.58557644104544...|    55.45|
|0050   |20130307|     40|[0.58671338876723...|     55.4|
|0050   |20130308|     41|[0.58725651096374...|     55.8|
|0050   |20130311|     42|[0.58933733603890...|     55.9|
|0050   |20130312|     43|[0.59115423865838...|    55.55|
|0050   |20130313|     44|[0.59086160641190...|    55.65|
+-------+--------+-------+--------------------+---------+
only showing top 10 rows

+-------+--------+-------+--------------------+---------+
| etf_id|etf_date|row_idx|         stdFeatures|etf_close|
+-------

In [20]:
#test_dt.show(20, False)

In [21]:
train_dt.cache()
test_dt.cache()

DataFrame[etf_id: string, etf_date: string, row_idx: int, stdFeatures: vector, etf_close: double]

In [22]:
#取出etf的distinct id
etf_ids = []
for row in test_dt.select("etf_id").distinct().collect():
    etf_ids.append(row["etf_id"])
etf_ids

['00701  ',
 '0051   ',
 '0057   ',
 '006203 ',
 '0052   ',
 '0050   ',
 '0055   ',
 '0054   ',
 '0059   ',
 '00690  ',
 '00713  ',
 '006204 ',
 '006208 ',
 '0053   ',
 '006201 ',
 '0056   ',
 '00692  ',
 '0058   ']

In [57]:
#訓練Model及評估(RandomForestRegressor - etf_id wise) 
#-- accuracy:0.48, (RMSE) on test data = 0.938023 --> using stdScaler
#-- accuracy:0.48, (RMSE) on test data = 0.87064 --> no Scaler
#-- accuracy:0.56, (RMSE) on test data = 0.92546 --> use minmaxScaler
#-- accuracy:0.513812 , (RMSE) on test data = 0.597397 --> use minmaxScaler and no Scaler(full data)
from pyspark.ml.regression import RandomForestRegressor

rf = RandomForestRegressor(featuresCol="stdFeatures",labelCol="etf_close", maxDepth=10, maxBins=30)
#記下對應etf_id的所有model
model_map = {}
for etfid in etf_ids:
    train_data = train_dt.filter("etf_id='" + etfid + "'")
    rf_model = rf.fit(train_data)
    model_map.update({etfid: rf_model})

In [58]:
#find last records of all etf to be the base of next record
tetf_dt_prod = tetfp_dt2.filter("etf_date < '" + predict_start_date + "' and MACD is not null")
tetf_max_idx = tetf_dt_prod.groupBy("etf_id").max("row_idx")
tetf_max = tetf_max_idx.select(col("etf_id"), col("max(row_idx)").cast("Double").alias("row_idx")) \
    .join(tetf_dt_prod, ["etf_id", "row_idx"], "inner") \
    .select("etf_id", "etf_date", "row_idx", "etf_close", "close_price_raw", "dif_list") \
    .orderBy('etf_id')

tetf_max.show()

+-------+--------+-------+---------+--------------------+--------------------+
| etf_id|etf_date|row_idx|etf_close|     close_price_raw|            dif_list|
+-------+--------+-------+---------+--------------------+--------------------+
|0050   |20180420| 1298.0|    80.75|[54.4, 54.85, 54....|[0.35882671868262...|
|0051   |20180420| 1298.0|    33.02|[26.09, 26.12, 26...|[0.23810818793990...|
|0052   |20180420| 1298.0|     54.8|[32.72, 32.12, 32...|[0.30196772810150...|
|0053   |20180420| 1298.0|     35.3|[23.26, 23.11, 22...|[0.15597485321855...|
|0054   |20180420| 1298.0|    23.85|[19.4, 19.37, 19....|[0.09327655672724...|
|0055   |20180420| 1298.0|    16.95|[11.47, 11.49, 11...|[0.31374969126963...|
|0056   |20180420| 1298.0|    25.82|[22.95, 23.06, 22...|[0.31828091002715...|
|0057   |20180420| 1298.0|    51.15|[31.94, 31.93, 31...|[0.20109705259728...|
|0058   |20180420| 1298.0|     46.1|[32.12, 31.96, 32...|[-0.1346882469799...|
|0059   |20180420| 1298.0|    41.79|[24.65, 24.67, 2

In [59]:
#建立產生feature之方法
def create_feature(orig_dt):
    new_dt = orig_dt.withColumn("EMA5", calculate_ema(col("close_price_raw"), lit(5))) \
    .withColumn("EMA10", calculate_ema(col("close_price_raw"), lit(10))) \
    .withColumn("EMA20", calculate_ema(col("close_price_raw"), lit(20))) \
    .withColumn("BIAS", calculate_bias(col("close_price_raw"))) \
    .withColumn("KD", calculate_KD(col("close_price_raw"))) \
    .withColumn("K", col("KD")[0]).withColumn("D", col("KD")[1]) \
    .withColumn("MACD", calculate_MACD(col("dif_list"), col("DIF"))) \
    .withColumn("RSI", calculate_RSI(col("close_price_raw")))\
    .withColumn("WR", calculate_WR(col("close_price_raw")))
    return new_dt
#計算上或下的值(udf)
def judge_up_down_pred(curr_price, close_price_list):
    if len(close_price_list) == 0:
        return 0.0
    else:
        prev_price = close_price_list[-1]
        if curr_price == prev_price:
            return 0.0
        elif curr_price > prev_price:
            return 1.0
        else:
            return 2.0
judge_up_down_pred=udf(judge_up_down_pred, DoubleType())
#建立預測方法
def doPredict(test_dt, etf_ids, model_dic):
    predict_res = None
    for etfid in etf_ids:
        test_data = test_dt.filter("etf_id='" + etfid + "'")
        predicts = model_dic[etfid].transform(test_data)
        if predict_res is None:
            predict_res = predicts
        else:
            predict_res = predict_res.unionAll(predicts)
    predict_res2 = predict_res.withColumn("pred_ud", judge_up_down_pred(col("prediction"), col("close_price_raw")))
    return predict_res2

In [60]:
schema = tetfp_dt2.select("etf_id", "etf_date", col("row_idx").cast("Double"), "close_price_raw", "DIF", "dif_list").schema
idx_plus = 0.0
predict_res_final = None
for date in next_date_range:
    next_rows = []
    idx_plus = idx_plus +1
    for row in tetf_max.collect():
        close_price_raw = row["close_price_raw"]
        close_price_raw.append(row["etf_close"])
        dif_list = row["dif_list"]
        dif = calculate_DIF_native(close_price_raw)
        dif_list.append(dif)
        next_row = (row["etf_id"], date, row["row_idx"]+idx_plus, close_price_raw, dif ,dif_list)
        next_rows.append(next_row)
    orig_dt = spark.createDataFrame(next_rows, schema)
    new_dt = create_feature(orig_dt) #加入預測用的feature
    feature_dt = assembler.transform(new_dt).withColumn("stdFeatures", col("features")) #作出 FeatureVector
    #feature_dt.select("etf_id", "etf_date", "stdFeatures").show(10, False)
    #do predict using existed model
    pred_res = doPredict(feature_dt, etf_ids, model_map)
    #取出所有預測結果作合併，以進行後續成績計算
    merged_dt = pred_res.select("etf_id", "etf_date", "prediction", "pred_ud")
    if predict_res_final is None:
        predict_res_final = merged_dt
    else:
        predict_res_final = predict_res_final.unionAll(merged_dt)
    tetf_max = pred_res.select("etf_id", "etf_date", "row_idx", col("prediction").alias("etf_close"), "close_price_raw", "dif_list")
    
predict_res_final.show()

+-------+--------+------------------+-------+
| etf_id|etf_date|        prediction|pred_ud|
+-------+--------+------------------+-------+
|00701  |20180423|20.779500000000006|    2.0|
|0051   |20180423|31.600000000000005|    2.0|
|0057   |20180423|50.331999999999994|    2.0|
|006203 |20180423|             37.56|    2.0|
|0052   |20180423| 53.90749999999999|    2.0|
|0050   |20180423| 82.33250000000001|    1.0|
|0055   |20180423|16.566999999999997|    2.0|
|0054   |20180423|            25.015|    1.0|
|0059   |20180423|           40.9195|    2.0|
|00690  |20180423|22.029000000000003|    1.0|
|00713  |20180423|           30.4125|    2.0|
|006204 |20180423| 52.62550000000001|    2.0|
|006208 |20180423|47.952999999999996|    1.0|
|0053   |20180423|           37.1845|    1.0|
|006201 |20180423|            14.162|    2.0|
|0056   |20180423|25.669999999999998|    2.0|
|00692  |20180423|            21.076|    2.0|
|0058   |20180423|            47.122|    1.0|
|00701  |20180424|20.7080000000000

In [61]:
#評估Model的RMES
####### with min-max scaler (depth=16, bin=48) #########
#Root Mean Squared Error (RMSE) on test data = 1.66686   accuracy = 0.544444 
####### with min-max scaler (depth=15, bin=40) #########
#Root Mean Squared Error (RMSE) on test data = 1.62652   accuracy = 0.54444 
####### with min-max scaler (depth=10, bin=30) #########
#Root Mean Squared Error (RMSE) on test data = 1.43988   accuracy = 0.511111 
####### with min-max scaler (default rf param) #########
#Root Mean Squared Error (RMSE) on test data = 1.57899   accuracy = 0.488889 
####### with min-max scaler (depth=10, bin=32) #########
#Root Mean Squared Error (RMSE) on test data = 1.56144   accuracy = 0.477778 
####### with min-max scaler (bin=20) #########
#Root Mean Squared Error (RMSE) on test data = 1.67822   accuracy = 0.455556 
####### with std scaler (default rf param) #########
#Root Mean Squared Error (RMSE) on test data = 1.7876   accuracy = 0.3333 
####### withoutscaler (default rf param) #########
#Root Mean Squared Error (RMSE) on test data = 1.12874   accuracy = 0.411111 

In [62]:
from pyspark.ml.evaluation import RegressionEvaluator, MulticlassClassificationEvaluator
tetf_dt_eval = tetfp_dt2.filter("etf_date >= '" + predict_start_date + "'") \
    .select("etf_id", "etf_date", "etf_close", "close_price_raw")
eval_dt = predict_res_final.join(tetf_dt_eval, ["etf_id", "etf_date"], "inner") \
    .withColumn("act_ud", judge_up_down_pred(col("etf_close"), col("close_price_raw")))
eval_dt.orderBy("etf_id", "etf_date", ascending=True).show(10)

evaluator = RegressionEvaluator(
    labelCol="etf_close", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(eval_dt)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)
evaluator = MulticlassClassificationEvaluator(
    labelCol="act_ud", predictionCol="pred_ud", metricName="accuracy")
accuracy = evaluator.evaluate(eval_dt)
print("accuracy = %g " % accuracy)

+-------+--------+------------------+-------+---------+--------------------+------+
| etf_id|etf_date|        prediction|pred_ud|etf_close|     close_price_raw|act_ud|
+-------+--------+------------------+-------+---------+--------------------+------+
|0050   |20180423| 82.33250000000001|    1.0|    79.95|[54.4, 54.85, 54....|   2.0|
|0050   |20180424|           82.9075|    1.0|    79.55|[54.4, 54.85, 54....|   2.0|
|0050   |20180425|           83.7175|    1.0|     79.3|[54.4, 54.85, 54....|   2.0|
|0050   |20180426|             84.51|    1.0|    79.05|[54.4, 54.85, 54....|   2.0|
|0050   |20180427|            84.685|    1.0|     79.2|[54.4, 54.85, 54....|   1.0|
|0051   |20180423|31.600000000000005|    2.0|     33.0|[26.09, 26.12, 26...|   2.0|
|0051   |20180424|31.263499999999993|    2.0|    32.44|[26.09, 26.12, 26...|   2.0|
|0051   |20180425|31.078999999999997|    2.0|    32.02|[26.09, 26.12, 26...|   2.0|
|0051   |20180426|            30.915|    2.0|    31.71|[26.09, 26.12, 26...|

In [66]:
#define method to evaluate by ETF way
final_res = eval_dt.select("etf_id", "etf_date", "prediction", "pred_ud", "etf_close", "act_ud") \
    .orderBy("etf_date", "etf_id").collect()
weights = [0.1, 0.15, 0.2, 0.25, 0.3]
judge_score = 0.0
for row in final_res:
    #(up equal*0.5+((act_p - abs(pred_p - act_p))/act_p)*0.5)*weight
    act_price = row["etf_close"]
    pred_price = row["prediction"]
    eidx = next_date_range.index(row["etf_date"])
    judge_score = judge_score + \
        ((0.5 if row["pred_ud"] == row["act_ud"] else 0) + \
         ((act_price-abs(pred_price-act_price))/act_price)*0.5)*weights[eidx]
print("final score: ", judge_score)

final score:  13.411485648426304


In [70]:
#export to pandas
etf_ids = []
mon_ud = []
mon_price = [] 
tue_ud = []
tue_price = []
wed_ud = []
wed_price = []
thu_ud = []
thu_price = []
fri_ud = []
fri_price = []

def encode_ud(oper_ud):
    if oper_ud == 0.0:
        return 0
    elif oper_ud == 1.0:
        return 1
    else:
        return -1

for row in final_res:
    etf_id = row["etf_id"]
    if etf_id not in etf_ids:
        etf_ids.append(etf_id)
    etf_date = row["etf_date"]
    eidx = next_date_range.index(row["etf_date"])
    if eidx == 0:
        mon_ud.append(encode_ud(row["pred_ud"]))
        mon_price.append(row["prediction"])
    elif eidx == 1:
        tue_ud.append(encode_ud(row["pred_ud"]))
        tue_price.append(row["prediction"])
    elif eidx == 2:
        wed_ud.append(encode_ud(row["pred_ud"]))
        wed_price.append(row["prediction"])
    elif eidx == 3:
        thu_ud.append(encode_ud(row["pred_ud"]))
        thu_price.append(row["prediction"])
    elif eidx == 4:
        fri_ud.append(encode_ud(row["pred_ud"]))
        fri_price.append(row["prediction"])
dic = {"ETFid": etf_ids, 
       "Mon_ud": mon_ud, "Mon_cprice": mon_price,
       "Tue_ud": tue_ud, "Tue_cprice": tue_price,
       "Wed_ud": wed_ud, "Wed_cprice": wed_price,
       "Thu_ud": thu_ud, "Thu_cprice": thu_price,
       "Fri_ud": fri_ud, "Fri_cprice": fri_price
      }
final_df = pd.DataFrame(data=dic)[['ETFid','Mon_ud','Mon_cprice','Tue_ud','Tue_cprice',
                                  'Wed_ud','Wed_cprice','Thu_ud','Thu_cprice',
                                  'Fri_ud','Fri_cprice']]
final_df

,ETFid,Mon_ud,Mon_cprice,Tue_ud,Tue_cprice,Wed_ud,Wed_cprice,Thu_ud,Thu_cprice,Fri_ud,Fri_cprice
0,0050,1,82.3325,1,82.9075,1,83.7175,1,84.5100,1,84.6850
1,0051,-1,31.6000,-1,31.2635,-1,31.0790,-1,30.9150,-1,30.5970
2,0052,-1,53.9075,-1,53.6100,1,53.8950,1,53.9925,0,53.9925
3,0053,1,37.1845,1,37.2725,1,37.3755,1,37.5515,1,37.6730
4,0054,1,25.0150,1,25.3730,-1,25.3630,1,25.4105,1,25.4200
5,0055,-1,16.5670,-1,16.4720,-1,16.4215,-1,16.3095,-1,16.0750
6,0056,-1,25.6700,-1,25.6225,-1,25.6060,-1,25.5620,-1,25.5330
7,0057,-1,50.3320,-1,49.8770,-1,49.3385,-1,48.8850,-1,48.7105
8,0058,1,47.1220,1,47.8075,0,47.8075,1,47.8960,-1,47.8760
9,0059,-1,40.9195,-1,40.5705,-1,40.3515,-1,40.0795,-1,39.9165


In [63]:
#ll = [46.92, 47.31, 47.0, 46.79, 46.49, 46.66, 47.0, 46.96, 47.0]
ll = [46.92, 47.31, 47.0, 46.79, 46.49, 46.66]
win_len=5
print(ll[0])
print(ll[0: -1])
print(list(map(lambda x,y : x - y, ll[1:], ll[0: -1])))
for x in ll[win_len:]:
    print(x)
ema = sum(ll[:win_len])/len(ll[:win_len])
print(ema)
for price in ll[win_len:]:
    ema = (ema*(win_len-1)+price*2)/(win_len+1)
tup1, tup2 = (1,2)
print(tup1, ' ', tup2)
tup = (3,4)
print(tup[0], ' ', tup[1])
list(range(0,2))
test_dic = {}
test_dic.update({"0051": assembler})
print(test_dic['0051'])

46.92
[46.92, 47.31, 47.0, 46.79, 46.49]
[0.39000000000000057, -0.3100000000000023, -0.21000000000000085, -0.29999999999999716, 0.1699999999999946]
46.66
46.902
1   2
3   4
VectorAssembler_43d39044b48e02423b41
